In [28]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import argparse
import subprocess
import os
from scipy.spatial import cKDTree
from netCDF4 import Dataset, default_fillvals
import xarray as xr
import xarray.ufuncs as xu


In [3]:
# import NetCDF params file
direc = '/Users/diana/Dropbox/UW/Research/rasm/25_km'
ncparams_50km = 'vic_params_joe_copy.nc'
params_50km = xr.open_dataset(os.path.join(direc, ncparams_50km))
domain_25km = xr.open_dataset(os.path.join(direc, 'domain.lnd.wr25b_ar9v4.170413.nc'))

In [27]:
domain_25km.xc

<xarray.DataArray 'xc' (nj: 413, ni: 551)>
array([[-170.607248, -170.524011, -170.440378, ...,  -66.122572,  -65.999494,
         -65.876997],
       [-170.733438, -170.650354, -170.566888, ...,  -65.985853,  -65.862836,
         -65.740408],
       [-170.859995, -170.777063, -170.693765, ...,  -65.848539,  -65.725599,
         -65.603239],
       ..., 
       [ 123.609935,  123.52798 ,  123.445659, ...,   17.030948,   16.90845 ,
          16.786548],
       [ 123.481227,  123.399104,  123.3166  , ...,   17.17176 ,   17.049175,
          16.927192],
       [ 123.35287 ,  123.27058 ,  123.187908, ...,   17.311965,   17.189304,
          17.067237]])
Coordinates:
    xc       (nj, ni) float64 -170.6 -170.5 -170.4 -170.4 -170.3 -170.2 ...
    yc       (nj, ni) float64 16.25 16.38 16.5 16.62 16.74 16.86 16.98 17.1 ...
Dimensions without coordinates: nj, ni
Attributes:
    long_name:  longitude of grid cell center
    units:      degrees_east
    bounds:     xv

In [5]:
params_50km.lons.max()

<xarray.DataArray 'lons' ()>
array(179.9969)

In [6]:
def lon_lat_to_cartesian(lon, lat, radius=1):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius radius
    """

#     # Unpack xarray object into plane arrays
#     if hasattr(lon, 'data'):
#         lon = lon.data
#     if hasattr(lat, 'data'):
#         lat = lat.data

#     if lon.ndim != lat.ndim:
#         raise ValueError('coordinate must share the same number of dimensions')

#     if lon.ndim == 1:
#         lon, lat = np.meshgrid(lon, lat)

    lon_r = xu.radians(lon)
    lat_r = xu.radians(lat)

    x = radius * xu.cos(lat_r) * xu.cos(lon_r)
    y = radius * xu.cos(lat_r) * xu.sin(lon_r)
    z = radius * xu.sin(lat_r)

    return x.flatten(), y.flatten(), z.flatten()

In [7]:
def latlon2yx(plons, plats, glons, glats):
    '''find y x coordinates'''

    if glons.ndim == 1 or glats.ndim == 1:
        glons, glats = np.meshgrid(glons, glats)

    px, py, pz = lon_lat_to_cartesian(plons, plats)
    gx, gy, gz = lon_lat_to_cartesian(glons, glats)

    combined = np.dstack(([gx, gy, gz]))[0]
    points = list(np.vstack((px, py, pz)).transpose())

    mytree = cKDTree(combined)
    dist, indexes = mytree.query(points, k=1)
    y, x = np.unravel_index(np.array(indexes), glons.shape)
    return y, x

exclude lats/lngs that aren't in land mask and get indices for the land gridcells 

In [8]:
# 25km land lats/lngs
yc_land_25km = domain_25km.yc.where(domain_25km.mask == 1, drop=True)
xc_land_25km = domain_25km.xc.where(domain_25km.mask == 1, drop=True)
land_lats_25km = yc_land_25km.values[~np.isnan(yc_land_25km.values)]
land_lons_25km = xc_land_25km.values[~np.isnan(xc_land_25km.values)]
nj_inds_25km, ni_inds_25km = np.nonzero(domain_25km.mask.values)

# 50km land lats/lngs
yc_land_50km = params_50km.yc.where(params_50km.run_cell == 1, drop=True)
xc_land_50km = params_50km.xc.where(params_50km.run_cell == 1, drop=True)
land_lats_50km = yc_land_50km.values[~np.isnan(yc_land_50km.values)]
land_lons_50km = xc_land_50km.values[~np.isnan(xc_land_50km.values)]
nj_inds_50km, ni_inds_50km = np.nonzero(params_50km.run_cell.values)

get indices for mapping back into 50km

## NOTE: plats/plons will come from the 50km grid, the glons/glats will be from the 25km grid 

In [9]:
# y_inds, x_inds = latlon2yx(land_lons_50km, land_lats_50km, land_lons_25km, land_lats_25km)

try using `scipy.ndimage.zoom` instead

In [13]:
from scipy import ndimage
# ten_by_ten = ndimage.zoom(five_by_five, 2, order=0)
u = ndimage.zoom(params_50km.mask.values, 2, order=0)

In [18]:
domain_25km

<xarray.Dataset>
Dimensions:  (ni: 551, nj: 413, nv: 4)
Coordinates:
    xc       (nj, ni) float64 -170.6 -170.5 -170.4 -170.4 -170.3 -170.2 ...
    yc       (nj, ni) float64 16.25 16.38 16.5 16.62 16.74 16.86 16.98 17.1 ...
Dimensions without coordinates: ni, nj, nv
Data variables:
    xv       (nj, ni, nv) float64 -170.6 -170.5 -170.6 -170.7 -170.5 -170.4 ...
    yv       (nj, ni, nv) float64 16.15 16.27 16.35 16.23 16.27 16.4 16.48 ...
    mask     (nj, ni) int32 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    area     (nj, ni) float64 6.398e-06 6.417e-06 6.437e-06 6.458e-06 ...
    frac     (nj, ni) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
Attributes:
    title:               CESM domain data:
    Conventions:         CF-1.0
    source_code:         SVN $Id: gen_domain.F90 41914 2012-11-13 21:58:37Z m...
    SVN_url:              $URL: https://svn-ccsm-models.cgd.ucar.edu/tools/ma...
    Compiler_Optimized:  FALSE
    hostname:            geyser03
    history

In [22]:
domain_25km['mask_from_params'] = domain_25km.mask

In [25]:
# domain_25km['mask_from_params'].values = u
print(domain_25km['mask_from_params'].shape)
print(u.shape)

(413, 551)
(410, 550)
